# Regras de Associação com Apriori — Exemplo de Supermercado

In [ ]:
# Instalar a biblioteca mlxtend (caso ainda não esteja instalada)
import warnings
warnings.filterwarnings('ignore')

!pip install mlxtend

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Carregando bibliotecas

In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

##Criar transações simuladas de supermercado

In [ ]:
# Lista de transações (cada sublista representa uma compra)
transacoes = [
    ['leite', 'pão', 'manteiga'],
    ['leite', 'pão'],
    ['pão', 'manteiga'],
    ['leite', 'manteiga'],
    ['leite', 'pão', 'manteiga', 'cerveja'],
    ['pão', 'cerveja'],
    ['leite', 'cerveja'],
    ['pão', 'manteiga'],
    ['leite', 'pão', 'manteiga'],
    ['cerveja', 'batata']
]

## Transformar dados em formato binário


In [ ]:
# Codificar as transações como matriz booleana
te = TransactionEncoder()
te_array = te.fit(transacoes).transform(transacoes)

# Criar DataFrame binário
df = pd.DataFrame(te_array, columns=te.columns_)
df.head()

,batata,cerveja,leite,manteiga,pão
0,False,False,True,True,True
1,False,False,True,False,True
2,False,False,False,True,True
3,False,False,True,True,False
4,False,True,True,True,True


## Aplicar o Algoritmo Apriori

In [ ]:
# Gerar os conjuntos frequentes com suporte mínimo de 30%
itemsets_frequentes = apriori(df, min_support=0.3, use_colnames=True)

# Visualizar os conjuntos mais frequentes
itemsets_frequentes.sort_values(by='support', ascending=False)

,support,itemsets
3,0.7,(pão)
1,0.6,(leite)
2,0.6,(manteiga)
6,0.5,"(pão, manteiga)"
0,0.4,(cerveja)
4,0.4,"(leite, manteiga)"
5,0.4,"(leite, pão)"
7,0.3,"(leite, pão, manteiga)"


## Gerar regras de associação

In [ ]:
# Gerar regras com confiança mínima de 70%
regras = association_rules(itemsets_frequentes, metric="confidence", min_threshold=0.7)

# Visualizar as regras mais relevantes
regras[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by='lift', ascending=False)

,antecedents,consequents,support,confidence,lift
2,"(leite, pão)",(manteiga),0.3,0.750000,1.250000
1,(manteiga),(pão),0.5,0.833333,1.190476
0,(pão),(manteiga),0.5,0.714286,1.190476
3,"(leite, manteiga)",(pão),0.3,0.750000,1.071429


## Interpretando as regras

In [ ]:
# Mostrar as 3 regras mais fortes
for i, row in regras.sort_values(by='lift', ascending=False).head(3).iterrows():
    antecedente = ', '.join(list(row['antecedents']))
    consequente = ', '.join(list(row['consequents']))
    print(f"Se o cliente comprou [{antecedente}], então provavelmente comprou [{consequente}] → confiança: {row['confidence']:.2f}, lift: {row['lift']:.2f}")

Se o cliente comprou [leite, pão], então provavelmente comprou [manteiga] → confiança: 0.75, lift: 1.25
Se o cliente comprou [manteiga], então provavelmente comprou [pão] → confiança: 0.83, lift: 1.19
Se o cliente comprou [pão], então provavelmente comprou [manteiga] → confiança: 0.71, lift: 1.19


- O algoritmo Apriori identificou padrões como:
- “Se o cliente comprou pão, há alta chance de ter comprado manteiga.”

- Métricas importantes:
- Confiança: probabilidade da regra se confirmar
- Lift: impacto da regra (quanto maior que 1, mais forte a associação)

- Esse tipo de análise é útil para:
- Organização de prateleiras
- Promoções combinadas
- Estratégias de marketin
